In [91]:
class SimpleClass():
    def __init__(self, name):
        print('hell0 ' + name)
        
    def yell(self):
        print('YELLING')
        

In [92]:
s = 'world'

In [93]:
x = SimpleClass('Jakk')

hell0 Jakk


In [94]:
x.yell()

YELLING


In [95]:
class ExtendedClass(SimpleClass):
    def __init__(self, name):
        super().__init__(name)
        print('Extend!!!')

In [96]:
y = ExtendedClass('JAKKRIT')
y.yell()

hell0 JAKKRIT
Extend!!!
YELLING


In [97]:
y.yell()

YELLING


In [98]:
class Operation():
    def __init__(self, input_nodes=[]):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
            
    def compute(self):
        pass

In [99]:
class add(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [100]:
class multiply(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var * y_var

In [101]:
class matrixMultiply(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var.dot(y_var)

In [102]:
class Placeholder():
    def __init__(self):
        self.output_nodes = []
        _default_graph.placeholders.append(self)

In [103]:
class Variable():
    def __init__(self, initial_value = None):
        self.value = initial_value
        self.output_nodes = []
        _default_graph.variables.append(self)

In [112]:
class Graph():
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
    
    def set_as_default(self):
        global _default_graph
        _default_graph = self

In [113]:
def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
            nodes_postorder.append(node)
            
    recurse(operation)
    return nodes_postorder

In [114]:
class Session():
    def run(self, operation, feed_dict={}):
        nodes_postorder = traverse_postorder(operation)
        for node in nodes_postorder:
            if type(node) == Placeholder:
                node.output = feed_dict[node]
            elif type(node) == Variable:
                node.output = node.value
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
            if type(node.output) == list:
                node.output = np.array(node.output)
            return operation.output

In [115]:
g = Graph()

In [116]:
g.set_as_default()

In [119]:
A = Variable(10)
b = Variable(1)
x = Placeholder()
y = multiply(A, x)
z = add(y, b)

In [120]:
sess = Session()
result = sess.run(operation=z, feed_dict={x:10})

AttributeError: 'Variable' object has no attribute 'output'